## Object Detection Using SSD Algorithm

### Importing libraries required for Object Detection using SSD

In [ ]:
from imutils.video import FPS
import numpy as np
import imutils
import cv2

### Defining some constants and Classes array. Our SSD model is trained on these 21 classes

In [ ]:
use_gpu = True
live_video = False
confidence_level = 0.5
fps = FPS().start()
ret = True
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
           "sofa", "train", "tvmonitor"]

### Defining colors array where each class is randomly assigned a color

In [ ]:
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

### Reading the network in a variable called net using cv2.dnn.readNetFromCaffe

In [ ]:
net = cv2.dnn.readNetFromCaffe('ssd_files/MobileNetSSD_deploy.prototxt', 'ssd_files/MobileNetSSD_deploy.caffemodel')

### the parameter use_gpu is set to TRUE, set the backend and target to Cuda

In [ ]:
if use_gpu:
    print("[INFO] setting preferable backend and target to CUDA...")
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

### Initialize the VideoCapture object either with 0 for live video or with the video file name

In [ ]:
print("[INFO] accessing video stream...")
if live_video:
    vs = cv2.VideoCapture(0)
else:
    vs = cv2.VideoCapture('test.mp4')

### in the infinite array and read the frames
##### if ret says that if the VideoCapture object is returning True, then only proceed
##### Resize the frame and get its height and width

In [ ]:
while ret:
    ret, frame = vs.read()
    if ret:
        frame = imutils.resize(frame, width=400)
        (h, w) = frame.shape[:2]

##### Create a blob from the image, set it as input, and pass it forward through the network using cv2.blobFromImage.

In [ ]:
blob = cv2.dnn.blobFromImage(frame, 0.007843, (300, 300), 127.5)
        net.setInput(blob)
        detections = net.forward()

##### traverse in the detections we got.
##### check the confidence of each and every detection.
##### the confidence is greater than a threshold then proceed.
##### Calculate the coordinates of the box and convert them to int

In [ ]:
for i in np.arange(0, detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > confidence_level:
                idx = int(detections[0, 0, i, 1])
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")

##### Drawing the rectangle around the found object.
##### Finally putting this label onto the original frame.

In [ ]:
label = "{}: {:.2f}%".format(CLASSES[idx], confidence * 100)
                cv2.rectangle(frame, (startX, startY), (endX, endY), COLORS[idx], 2)

                y = startY - 15 if startY - 15 > 15 else startY + 15
                cv2.putText(frame, label, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)

##### Show final results.
##### Break if someone hits the ESC key.
##### Update the fps counter.
##### Stop the fps counter.

In [ ]:
cv2.imshow('Live detection',frame)

        if cv2.waitKey(1)==27:
            break

        fps.update()

fps.stop()

##### Printing FPS metrics

In [ ]:
print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))